# Wprowadzenie do Spark RDD

## RDD = Resilient Distributed Datasets
Rozproszone obiekty RDD są reprezentacją rozproszonego zbioru danych.

Na obiektach RDD dozwolone są dwa typy operacji:

1. Transformacje: operacje, które tworzą nowe zbiory danych z istniejących RDD. 

Przykłady to `map`, `filter`, `flatMap`, `groupByKey`, `reduceByKey`, `sample`, `union`, `intersection`, `distinct`, `coalesce`, `repartition`.


2. Akcje: operacje, które zwracają wartość do sterownika programu po przetworzeniu danych.

Przykłady to `reduce`, `collect`, `count`, `first`, `take`, `takeSample`, `takeOrdered`, `saveAsTextFile`, `countByKey`, `foreach`.

W tym zeszycie skupimy się na podstawowych transformacjach i akcjach na obiektach RDD.


## Inicjowanie Sparka
Pracę ze Sparkiem zaczynamy od zainicjowania sesji Sparka.
Aby zainicjować Sparka, musimy zaimportować pakiet `findspark` i uruchomić metodę `init()`:

In [ ]:
import findspark
findspark.init() 

Następnie tworzymy obiekt sesji Sparka. Zwróć uwagę na ustawienie nazwy aplikacji Sparka: 

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# obiekt sesji zwykle ma nazwę "spark"
spark = SparkSession.builder.appName("DataScience").getOrCreate() 
spark

Mając obiekt sesji możemy wyciągnąć z niego tzw. kontekst Sparka, który pozwala na bezpośrednią pracę z kolekcjami obiektów RDD oraz podstawowymi usługami Sparka.

In [ ]:
sc = spark.sparkContext
sc

## Tworzenie obiektów RDD

W pierwszym ćwiczeniu spróbujemy stworzyć obiekty RDD z różnych kolekcji danych.

W tym celu wykonamy następujące kroki:
- utworzymy obiekt RDD z listy liczb
- utworzymy obiekt RDD z pliku tekstowego

### Utworzenie obiektu RDD z listy liczb

In [ ]:
rdd = sc.parallelize([1, 2, 3, 4, 5])       # parallelize() jest tranformacją, która tworzy obiekt RDD z listy
rdd

In [ ]:
rdd.collect()   # collect() jest akcją, która zwraca wartość obiektu RDD do sterownika programu

Transformacja `parallelize` przyjmuje nie tylko proste listy, ale również bardziej złożone struktury danych, takie jak listy zagnieżdżone, słowniki, itp.

In [ ]:
rdd = sc.parallelize([1, 2, [3, 4], 5, {"a": 1, "b": 2}])
rdd.collect()

Może przyjmować również inne kolekcje danych, np. generator:

In [ ]:
rdd = sc.parallelize(range(10))
rdd.collect()

### Utworzenie obiektu RDD z plików

RDD może czytać dane z plików w formatach binarnych oraz tekstowych. Każdy odczyt jest transformacją.

Poniżej użyliśmy textFile() do odczytania tekstu książki:

In [ ]:
book_rdd = sc.textFile("../../data/books/ulysses.txt") # textFile() jest tranformacją tworzącą obiekt RDD z pliku tekstowego
book_rdd


RDD nie posiada wbudowanych funkcji do czytania plików CSV czy JSON. Obróbkę można zrobić samemu lub skorzystać z zewnętrznych bibliotek. W praktyce korzysta się z modułu DataFrame. 

In [ ]:
stock_rdd = sc.textFile("../../data/stock/stock.csv")  # przeczyta plik CSV jak zwykły plik tekstowy
stock_rdd

## Podstawowe akcje na obiektach RDD
Akcja to operacja, która zwraca wartość do sterownika programu po przetworzeniu danych.

In [ ]:
rdd.collect()   # collect() jest akcją, która zwraca całą zawartość kolekcji RDD

In [ ]:
rdd.count()     # count() jest akcją, która zwraca ilość elementów obiektu RDD

In [ ]:
rdd.first()   # first() jest akcją zwracającą pierwszy element obiektu RDD

In [ ]:
rdd.take(5)     # take() jest akcją, która zwraca pierwsze N elementów obiektu RDD

In [ ]:
rdd.takeOrdered(5)      # takeOrdered() jest akcją, która zwraca N najmniejszych elementów obiektu RDD

In [ ]:
rdd.top(5)      # top() jest akcją, która zwraca N największych elementów obiektu RDD

In [ ]:
# to samo co takeOrdered(5) można uzyskać za pomocą dodatkowego argumentu w metodzie top()
rdd.top(5, key=lambda x: -x)      # top() jest akcją, która zwraca N najmniejszych elementów obiektu RDD

In [ ]:
rdd.takeSample(False, 5)                    # zwraca 5 losowych elementów obiektu RDD

In [ ]:
result = rdd.collect()  # collect() jest akcją, która zwraca wartość obiektu RDD do sterownika programu
result[:5]             # wyświetlenie pierwszych 5 elementów obiektu RDD - za pomocą wbudowanej operacji języka Python

## Akcje agregujące

Akcje agregujące to operacje, które łączą elementy obiektu RDD za pomocą funkcji.
Przykłady to `reduce`, `fold`, `aggregate`, `countByKey`, `countByValue`, `sum`, `mean`, `max`, `min`, `stdev`, `variance`.

In [ ]:
rdd = sc.parallelize([1, 2, 3, 4, 5])

### reduce()
reduce() ma za zadanie zredukować elementy obiektu RDD do jednego elementu za pomocą funkcji.

Przykładem może być suma lub maksimum elementów w RDD. Kluczową rzeczą jest to, że funkcja przekazywana do reduce musi być przemienna (wynik operacji nie zależy od kolejności argumentów) oraz łączna (wynik jest taki sam niezależnie od grupowania operacji).


In [ ]:
rdd.reduce(lambda x, y: x + y)

In [ ]:
rdd.reduce(lambda x, y: x * y)

### fold()

fold() działa podobnie do reduce(), ale wymaga wartości początkowej.

 Wartość początkowa jest używana jako początkowy wynik oraz jako argument dla operacji w przypadku pustych partycji RDD. To sprawia, że fold jest bardziej ogólna niż reduce. 
 Wartość początkowa oraz funkcja użyta w fold również muszą być przemienne i łączne, aby zagwarantować poprawność wyniku w rozproszonym środowisku obliczeniowym.
 

In [ ]:
rdd.fold(0, lambda x, y: x + y)

In [ ]:
rdd.fold(1, lambda x, y: x * y)

### aggregate()

aggregate() jest bardziej ogólna niż reduce() i fold(). Pozwala na zdefiniowanie dwóch funkcji: funkcji agregującej oraz funkcji łączącej.

Funkcja agregująca jest wywoływana na każdej partycji obiektu RDD i zwraca wartość typu U. Funkcja łącząca łączy wyniki funkcji agregującej z różnych partycji.



In [ ]:
rdd.aggregate((0, 0), 
              (lambda acc, value: (acc[0] + value, acc[1] + 1)), 
              (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])))


In [ ]:
rdd.aggregate((1, 1), 
              (lambda acc, value: (acc[0] * value, acc[1] + 1)), 
              (lambda acc1, acc2: (acc1[0] * acc2[0], acc1[1] + acc2[1]))
             )

### countByKey()

countByKey() zlicza liczbę wystąpień każdego klucza w obiekcie RDD, który jest zbiorem par klucz-wartość.


In [ ]:
rdd = sc.parallelize([("a", 1), ("b", 2), ("a", 3), ("b", 4), ("c", 5)])
rdd.countByKey()

### countByValue()

countByValue() zlicza liczbę wystąpień każdej wartości w obiekcie RDD.

In [ ]:
rdd = sc.parallelize([1, 2, 3, 4, 5, 1, 2, 3, 4, 5])
rdd.countByValue()

### sum()

sum() zwraca sumę elementów obiektu RDD.


In [ ]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.sum()

### mean(), max(), min(), stdev(), variance()

mean(), max(), min(), stdev(), variance() zwracają odpowiednio średnią, maksimum, minimum, odchylenie standardowe oraz wariancję elementów obiektu RDD.

In [ ]:
rdd.mean()

In [ ]:
rdd.max()

In [ ]:
rdd.min()

## Transformacje na obiektach RDD

Transformacje to operacje, które tworzą nowe zbiory danych z istniejących RDD.

### map()

map() jest najbardziej podstawową transformacją. Przyjmuje funkcję, która jest stosowana do każdego elementu obiektu RDD i zwraca nowy obiekt RDD.


In [ ]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.map(lambda x: x * 2).collect()

In [ ]:
rdd.map(lambda x: x ** 2).collect()

### flatMap()

flatMap() działa podobnie do map(), ale zwraca listę wyników dla każdego elementu obiektu RDD.


In [ ]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.flatMap(lambda x: [x, x * 2]).collect()

### filter()

filter() zwraca nowy obiekt RDD, który zawiera tylko elementy spełniające warunek podany w funkcji.


In [ ]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.filter(lambda x: x % 2 == 0).collect()

### groupByKey()

groupByKey() grupuje elementy obiektu RDD według klucza.

In [ ]:
rdd = sc.parallelize([("a", 1), ("b", 2), ("a", 3), ("b", 4), ("c", 5)])
result = rdd.groupByKey().collect()
for key, value in result:
    print(key, list(value))

In [ ]:
rdd.groupByKey().mapValues(list).collect()

In [ ]:
rdd.groupByKey().mapValues(lambda x: sum(x)).collect()

In [ ]:
rdd.groupByKey().mapValues(lambda x: sum(x) / len(x)).collect()

### reduceByKey()

reduceByKey() działa podobnie do groupByKey(), ale zamiast grupować elementy, redukuje je za pomocą funkcji przekazanej jako argument.


In [ ]:
rdd = sc.parallelize([("a", 1), ("b", 2), ("a", 3), ("b", 4), ("c", 5)])
rdd.reduceByKey(lambda x, y: x + y).collect()

In [ ]:
rdd.reduceByKey(lambda x, y: x * y).collect()

### sortByKey()

sortByKey() sortuje elementy obiektu RDD według klucza.


In [ ]:
rdd = sc.parallelize([("b", 2), ("a", 1), ("c", 3)])
rdd.sortByKey().collect()

### join()

join() łączy dwa obiekty RDD na podstawie klucza.


In [ ]:
rdd1 = sc.parallelize([("a", 1), ("b", 2), ("c", 3)])
rdd2 = sc.parallelize([("a", 4), ("b", 5), ("c", 6)])
rdd1.join(rdd2).collect()


### union()

union() łączy dwa obiekty RDD w jeden obiekt RDD.

In [ ]:
rdd1 = sc.parallelize([1, 2, 3])
rdd2 = sc.parallelize([4, 5, 6])
rdd1.union(rdd2).collect()

### intersection()

intersection() zwraca obiekt RDD, który zawiera elementy wspólne dwóch obiektów RDD.

In [ ]:
rdd1 = sc.parallelize([1, 2, 3, 4, 5])
rdd2 = sc.parallelize([4, 5, 6, 7, 8])
rdd1.intersection(rdd2).collect()

### distinct()

distinct() zwraca obiekt RDD bez duplikatów.


In [ ]:
rdd = sc.parallelize([1, 2, 3, 4, 5, 1, 2, 3, 4, 5])
rdd.distinct().collect()

## Partycje

Partycje to podstawowa jednostka obliczeniowa w Sparku. Każda partycja to fragment danych, który jest przetwarzany przez pojedynczy wątek.
Transformacje na obiektach RDD są wykonywane na partycjach, a nie na pojedynczych elementach.

In [ ]:
rdd.getNumPartitions()   # zwraca ilość partycji obiektu RDD

Domyślnie, Spark tworzy tyle partycji, ile jest rdzeni w klastrze. Możemy zmienić liczbę partycji za pomocą metody `repartition()` lub `coalesce()`.
Różnica między nimi polega na tym, że `repartition()` zawsze tworzy nowe partycje, podczas gdy `coalesce()` może łączyć istniejące partycje.

In [ ]:
rdd = rdd.repartition(4)   # repartition() jest transformacją, która zmienia liczbę partycji obiektu RDD
rdd.getNumPartitions()

In [ ]:
rdd = rdd.coalesce(2)     # coalesce() jest transformacją, która zmienia liczbę partycji obiektu RDD
rdd.getNumPartitions()

## Zapisywanie obiektów RDD

Obiekty RDD można zapisać do plików w formatach tekstowym oraz binarnym.
Tworzony jest katalog, w którym zapisywane są pliki z danymi.


In [ ]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.saveAsTextFile("output")   # zapisuje obiekt RDD do pliku tekstowego

In [ ]:
rdd.saveAsPickleFile("output2")   # zapisuje obiekt RDD do pliku binarnego